In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from scipy.signal import savgol_filter
import seaborn as sb
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import os

In [2]:
class Datatable:
    def __init__(self,datadict,fasta_path,genbank_path):
        '''This class is an all-in-one function to 1) extract dictionary keys from run outputs 
        and 2) prepare necessary information for promoter region mining. For 2), we need both the
        cds_from_genome.fasta as well as the .gbff as datadict's IDs are only found in the fasta 
        but the fasta is extremely unparseable (too many edge cases). The genbank file comes in handy
        to extract descriptive gene info after identifying which features to keep using the fasta. '''
        
        # assign all the dictionary keys that came from the output of optimal_sensors run
        self.reps = datadict['reps'] # list of replicates used 
        self.ntimepts = datadict['ntimepts'] # original number of timepoints in dataset i.e. before subsetting for parameter fitting
        self.X = datadict['X'] # dataset used to fit model parameters
        self.newntimepts = datadict['newntimepts'] # number of timepoints in dataset used to fit model parameters
        self.transcriptIDs = datadict['transcriptIDs'] # all transcriptIDs of original dataset
        self.keepers = datadict['keepers'] # indicies of genes kept for parameter fitting
        self.keep_transcriptIDs = [self.transcriptIDs[i] for i in self.keepers] # transcriptIDs corresponding to used dataset
        self.A = datadict['A'] # learned model
        self.cd = datadict['cd'] # R^2 obtained over n-step prediction
        self.percent_nonzero_to_zero = datadict['percent_nonzero_to_zero'] # percent of nonzero elements that were set to zero during sparsification
        self.C = datadict['C'] # observer matrix
        self.opt_horizon = datadict['opt_horizon'] # horizon for optimization to obtain observer matrix
        self.sparseThresh = datadict['sparseThresh'] # mod of all elements of A less than sparseThresh were set to 0
        self.filter_method = datadict['filter_method'] # the method used for gene downselection 
        self.filterBeforeBS = datadict['filterBeforeBS'] # 'True' if downselection was done before background subtraction
        self.norm = datadict['norm'] # 'True' if the data was normalized
        
        # get records of the genes that we have kept for analysis
        # have to use cds_from_genome.fasta because this is the where the transcriptIDs came from (e.g. lcl|AM181176.4_cds_CAY53368.1_5775)
        self.fasta_records = list(SeqIO.parse(fasta_path,'fasta')) # full cds_from_genome fasta
        self.keep_fasta_records = [] # getting records of genes that we have used
        for tx in self.keep_transcriptIDs:
            for rec in self.fasta_records:
                if rec.name == tx:
                    self.keep_fasta_records.append(rec)
                    
        # match locus_tags in keep_records (from fasta) with tags in genbank to easily parse rest of cds' description 
        # can grab gene names and tags from the fasta description, other info need to grab from the genbank
        self.genes, self.locus_tags = [],[] 
        for rec in self.keep_fasta_records:
            rec_elems = [x.strip().strip(']') for x in rec.description.split(' [')]
            if 'gene=' in str(rec_elems): # sequence has gene name (e.g. gene=dnaA)
                self.genes.append(rec_elems[1][5:])
                self.locus_tags.append(rec_elems[2][10:])
            elif 'gene=' not in str(rec_elems): # sequence has no gene name, but has locus tag
                self.genes.append('N/A')
                self.locus_tags.append(rec_elems[1][10:])
        
        # now map the locus_tags to corresponding features in genbank, grab feature's proteins, locations, and strandedness
        gb_records = next(SeqIO.parse(genbank_path,'genbank'))
        self.proteins,self.locations = [],[] # each element of locations will be of length 3, 0:start,1:end,2:strand
        for tag in self.locus_tags:
            for feature in gb_records.features:
                if feature.type == 'CDS':
                    if feature.qualifiers['locus_tag'][0] == tag: 
                        self.proteins.append(feature.qualifiers['product'][0])
                        if feature.strand == 1: # 5' -> 3'
                            self.locations.append([feature.location.start.position,feature.location.end.position,1])
                        elif feature.strand == -1: # 3' -> 5' (complementary)
                            self.locations.append([feature.location.start.position,feature.location.end.position,-1])            
        
        # sort C in ascending order and get corresponding inds to sort other lists
        # recall that C is a 1xN dimensional vector that contains the observability score for each gene
        self.sorted_inds = self.C[:,0].argsort() # getting indices of C in ascending order
        self.sorted_C = self.C[self.sorted_inds] # sort C in ascnding order 
        self.sorted_transcriptIDs = [self.keep_transcriptIDs[i] for i in self.sorted_inds]
        self.sorted_genes = [self.genes[i] for i in self.sorted_inds]
        self.sorted_locus_tags = [self.locus_tags[i] for i in self.sorted_inds]
        self.sorted_proteins = [self.proteins[i] for i in self.sorted_inds]
        self.sorted_locations = [self.locations[i] for i in self.sorted_inds]

In [3]:
data_dir = 'data/'
results_dir = 'run-outputs/'
genbank_path = data_dir+'GCA_000009225.1_ASM922v1_genomic.gbff'
cdsFasta_path = data_dir+'GCA_000009225.1_ASM922v1_cds_from_genomic.fa'

data0 = Datatable(pickle.load(open(results_dir+'dump012_filterB4BS_DTW_IC0_m8.pickle','rb')),cdsFasta_path,genbank_path)
data1 = Datatable(pickle.load(open(results_dir+'dump012_filterAfterBS_DTW_IC0_m8.pickle','rb')),cdsFasta_path,genbank_path)

In [4]:
def plot_gene_traces(data,genes,tags,reps=[0,1,2],savedir='figures/',savefig=False,showfig=False):
    '''data is of shape n x m x r where n is #genes, m is #tps, r is # replicates.
        Make sure n is a multiple of 10. genes is a list with the gene names and tags is a list of locus_tags '''
    tspan = np.linspace(20,100,data.shape[1])
    nreps = len(reps)
    data = data[:,:,reps]
    ncols = 10
    nrows = int(data.shape[0]/ncols) 
    fig, axs = plt.subplots(nrows, ncols, figsize=(22,17));
    mt =  ['o--','d--','p--']
    repname = ['R1','R2','R3']
    colors = ['tab:blue','tab:orange','tab:green']
    count = 0
    for ax in axs.reshape(-1):
        ymin,ymax = np.min(data[count])-25,np.max(data[count])+25
        for rr in range(0,nreps):
            ax.plot(tspan,data[count,:,rr],mt[rr],lw=2,ms=5,color=colors[rr],label=repname[rr]);
            # add gene names or locus tags to subplot title
            if genes[count] == 'N/A':
                ax.set_title(tags[count],fontsize=12)
            else:
                ax.set_title(genes[count],fontsize=12)
        # Hide the right and top spines and make bottom and left axes thicker
        for axis in ['right','top']:
            ax.spines[axis].set_visible(False)
        for axis in ['bottom','left']:
            ax.spines[axis].set_linewidth(1.2)
        # increase ticklabel fontsizes
        ax.tick_params(axis='both', which='major', labelsize=12)
        ax.set_ylim([ymin,ymax])
        count+=1
        
    axs[-1,-1].legend(repname,bbox_to_anchor=(1.05, 1),loc='upper left',markerscale=2,fontsize=12,shadow=True)
    
    # Set common labels
    fig.text(0.5, -0.01, r'$Time \; (minutes)$', ha='center', va='center',fontsize=25)
    fig.text(-0.04, 0.5, r'$TPM_{BS}$', ha='center', va='center', rotation='vertical',fontsize=25)
        
    plt.tight_layout();
    if savefig:
        plt.savefig(savedir)
    if not showfig:
        plt.close()

def plot_eigvals(matrix):
    theta = np.linspace(0,20,100)
    L = np.linalg.eigvals(matrix)
    plt.figure();
    plt.plot(np.real(L),np.imag(L),'o');
    plt.plot(np.cos(theta),np.sin(theta),color='black',alpha=0.3)
    plt.axis('equal');

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(15,3));
ax[0].bar(np.arange(0,len(data0.C)),data0.C[:,0])
ax[0].bar(np.arange(0,len(data1.C)),data1.C[:,0])
ax[0].set_ylabel(r'$C_{gene}$',fontsize=16);
ax[0].set_xlabel(r'$gene$',fontsize=16);
ax[1].plot(data0.sorted_C);
ax[1].plot(data1.sorted_C);
ax[1].set_ylabel(r'$C_{gene}$',fontsize=16);
ax[1].set_xlabel(r'$gene$',fontsize=16);

In [ ]:
k = 100
save_dir = 'figures/data0.pdf'
plot_gene_traces(data0.X[data0.sorted_inds[-k:]],data0.sorted_genes[-k:],data0.sorted_locus_tags[-k:],\
                    reps=data0.reps,savedir=save_dir,savefig=False,showfig=True)

In [ ]:
k = 100
save_dir = 'figures/data1'
plot_gene_traces(data1.X[data1.sorted_inds[-k:]],data1.sorted_genes[-k:],data1.sorted_locus_tags[-k:],\
                    reps=data1.reps,savedir=save_dir,savefig=False,showfig=True)

#### Get the intergenic regions (or putative promoter regions) from the genbank (gbff) file

In [5]:
# Copyright(C) 2009 Iddo Friedberg & Ian MC Fleming
# Released under Biopython license. http://www.biopython.org/DIST/LICENSE
# Do not remove this comment
# https://biopython.org/wiki/Intergenic_regions
# heavily modded from the biopython wiki, but the structure comes from the author's above!
def get_interregions(genbank_path,intergene_length=1):
    ''' The regions between genes are termed intergenic regions. This function mines every one of
    those regions present in a genbank (.gbff) file iff the region has length >= intergene_length.
    Overlapping genes do not have intergenic regions and other regions are < intergene_length. These do not 
    get mined. The output can be saved as a fasta file with the intergenic regions and descriptions such as 
    location, strand, and unique ID'''

    seq_record = next(SeqIO.parse(open(genbank_path), 'genbank'))
    genome_size = len(seq_record)
    cds_list = []
    # Loop over the genome file, get the CDS features on each of the strands
    for feature in seq_record.features:
        if feature.type == "CDS":
            mystart = feature.location.start.position
            myend = feature.location.end.position
            strand = feature.strand
            if feature.strand == -1 or feature.strand == 1:
                cds_list.append((mystart,myend,strand))
            else:
                print("No strand indicated %d-%d. Assuming +\n" % (mystart, myend))
                cds_list.append((mystart, myend, 1))
    intergenic_records = []
    for i, pospair in enumerate(cds_list[1:]):
        # Compare current start position to previous end position
        # first take care of the case where the gene starts at position 0 
        if cds_list[i][0] == 0: # if start of cds is at position 0
            last_end = cds_list[-1][1]
            this_start = genome_size
            if this_start - last_end >= intergene_length:
                intergene_seq = seq_record.seq[last_end:this_start]
                intergenic_records.append(SeqRecord(intergene_seq,id="%s-ign-%d"%(seq_record.name,len(cds_list)),\
                        description="%d-%d"%(last_end+1,this_start),))
        # the rest of the regions 
        else:
            last_end = cds_list[i][1]
            this_start = pospair[0]
            if this_start - last_end >= intergene_length:
                intergene_seq = seq_record.seq[last_end:this_start]
                intergenic_records.append(SeqRecord(intergene_seq,id="%s-ign-%d"%(seq_record.name,i),\
                            description="%d-%d"%(last_end+1,this_start),))

    return intergenic_records

In [7]:
intergenic_records = get_interregions(genbank_path,intergene_length=50)
# Q: What should be the minimum intergene_length? 

doWrite = False
if doWrite:
    outpath = os.path.splitext(os.path.basename(genbank_path))[0] + "_ign.fa"
    SeqIO.write(intergenic_records, open('data/'+outpath, 'w'), 'fasta');

#### Given the gene's location, output the nearest promoter region. 

In [8]:
def get_promoterregions(datatable,intergenic_records,genbank_path,rel_dist_thresh=5000):
    '''datatable is a class object that contains gene names, locations, etc. instantiated with Datatable
    intergenic_records is essentially a fasta with the intergenic sequences (where promoters live) and 
    it is this functions job to, for every gene and its location in the datatable, grab the corresponding 
    promoter region from the intergenic_records. Can write the promoter sequences to a fasta after. Each 
    promoter sequences will have a unique id corresponding to the gene it was corresponding to. '''
    
    # first get the genome size
    genome_size = len(next(SeqIO.parse(open(genbank_path),'genbank')))
    
    promoter_records = []
    for i,location in enumerate(datatable.locations):
        
        strand = location[2] # '1' or '-1' for sense and antisense
        dist = np.inf
        
        if strand == 1: # only look upstream for intergenic region
            gene_start = location[0]
            for feature in intergenic_records:
                intergenic_location = [int(x) for x in feature.description.split('-')]
                intergenic_end = intergenic_location[1]
                new_dist = gene_start - intergenic_end
                rel_dist = genome_size - intergenic_end + gene_start
                if new_dist < 0: # Case 1 for strand 1, upstream constraint needs to be enforced using rel_dist
                    if rel_dist < rel_dist_thresh: # rel_dist b/w intergenic_end & gene_start within rel_dist_thresh to enforce upstream constraint  
                        if rel_dist < dist:
                            dist = rel_dist
                            promoter_seq = feature.seq
                            ign_start = intergenic_location[0]
                            ign_end = intergenic_location[1]
                elif new_dist >= 0: # Case 2 for strand 1, upstream constraint is naturally satisfied
                    if new_dist < dist:
                        dist = new_dist
                        promoter_seq = feature.seq
                        ign_start = intergenic_location[0]
                        ign_end = intergenic_location[1]   
            
        elif strand == -1: # only look downstream for intergenic region
            gene_end = location[1]
            for feature in intergenic_records:
                intergenic_location = [int(x) for x in feature.description.split('-')]
                intergenic_start = intergenic_location[0]
                new_dist = intergenic_start - gene_end
                rel_dist = genome_size - gene_end + intergenic_start
                if new_dist >= 0: # Case 1 for strand -1, downstream constraint is naturally satisfied
                    if new_dist < dist:
                        dist = new_dist
                        promoter_seq = feature.seq
                        ign_start = intergenic_location[0]
                        ign_end = intergenic_location[1]
                elif new_dist < 0: # Case 2 for strand -1, downstream constraint is naturally satisfied
                    if rel_dist < rel_dist_thresh:
                        if rel_dist < dist:
                            dist = rel_dist
                            promoter_seq = feature.seq
                            ign_start = intergenic_location[0]
                            ign_end = intergenic_location[1]
        
        promoter_records.append(SeqRecord(promoter_seq,id="%s_%d"%(datatable.locus_tags[i],i),\
                    description="%d-%d"%(ign_start,ign_end),name="%s%s"%('Promoter sequence for ',datatable.locus_tags[i]),))

    return promoter_records



In [10]:
promoter_records = get_promoterregions(data0,intergenic_records,genbank_path,rel_dist_thresh=5000)

doWrite = False
if doWrite:
    outpath = os.path.splitext(os.path.basename(genbank_path))[0] + "_promoters.fa"
    SeqIO.write(promoter_records, open('data/'+outpath, 'w'), 'fasta');